In [1]:
import numpy as np
import cv2 as cv
import datetime
from sklearn import cluster,datasets
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from datetime import datetime, timedelta

In [2]:
def file_reading(file_path):
    #reading annotations file
    ann_file = open(file_path,"r") #opening file in read mode only
    strings = [x.strip() for x in ann_file.readlines()]
    stimes=[]
    etimes=[]
    for i in range(len(strings)):
        s1,s2=strings[i].split("-")
        stimes.append(s1.strip())
        etimes.append(s2.strip())
    return stimes,etimes

In [3]:
def compare(stime,etime,abnormal_stimes,abnormal_etimes):
    length = len(abnormal_stimes)
    for i in range(length):
        t1 = datetime.strptime(abnormal_stimes[i], '%M:%S').time()
        t2 = datetime.strptime(abnormal_etimes[i], '%M:%S').time()
        obj1 = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
        obj2 = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
        if (stime >= obj1 and etime <= obj2) or (stime < obj1 and etime > obj1) or (stime < obj2 and etime > obj2):
            return 1
    return -1

In [4]:
class labeling_objects:
    def __init__(self,clip_no,stime,etime,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.label = label

In [5]:
def video_input(path,ann_file_path):
    cap = cv.VideoCapture(path)
    ret, frame1 = cap.read()

    #reading first frame
    prvs_gray = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    width=prvs_gray.shape[1]
    height = prvs_gray.shape[0]

    prvs = cv.resize(prvs_gray,(int(width/12),int(height/12)),interpolation = cv.INTER_AREA)
    print("previous shape ",prvs.shape)
    
    #intializing all values
    cnt=0
    flow_array = []
    mag_list = []
    flow_array_array = [] #for storing all clips
    secs = 0
    clip = 0
    label_objects_array = []
    abnormal_stimes,abnormal_etimes = file_reading(ann_file_path)
    
    while(True):
        ret, frame2 = cap.read()
        cnt=cnt+1
        if cnt%50 == 0:
            flow_array_array.append(flow_array)
            flow_array=[]
            clip+=1
            #adding labels here after one clip is recorded.
            secs = secs+2
            stime = timedelta(seconds = secs-2)
            etime = timedelta(seconds = secs)
            label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
            label_objects_array.append(labeling_objects(clip,stime,etime,label))
            
        if ret==False:
            break

        #converting frame into gray and resizing 
        gray2 = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        next = cv.resize(gray2,(int(width/12),int(height/12)),interpolation = cv.INTER_AREA)

        #calculating optical flow giving two consecutive frames as input
        flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # appending to the flow array
        flow_array.append(flow)

        #changing current frame as previous frame
        prvs = next
    if len(flow_array)!= 0:
        flow_array_array.append(flow_array)
        clip+=1
        secs = secs+1
        stime = timedelta(seconds = secs-1)
        etime = timedelta(seconds = secs)
        label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
        label_objects_array.append(labeling_objects(clip,stime,etime,label))
    flow_array_length = len(flow_array_array)
    print("Total no of clips",flow_array_length)
    print("total frames",cnt)

    mag_arr_all_clips=[]
    for j in range(flow_array_length):
        mag_list=[]
        flow_array=flow_array_array[j]
        for i in range(len(flow_array)):
            mag_list.append(cv.cartToPolar(flow_array[i][...,0],flow_array[i][...,1])[0])
        mag_arr = np.asarray(mag_list)
        mag_arr_all_clips.append(mag_arr)
    mag_arr_all_clips = np.asarray(mag_arr_all_clips)
    
    return mag_arr_all_clips,label_objects_array

In [6]:
#intial parameters for em clustering 
def intial_parameters(arr):
    k=4
    intial_weights = [(1-0.1)/4 for i in range(k)]
    intial_weights = np.append(intial_weights,0.1) # adding extra weight element for background purpose
    means = np.random.choice(arr.flatten(),k)
    means =np.append(means,0) # adding extra mean element for background purpose
    cov = np.random.sample(size=k)
    cov = np.append(cov,4.0) # adding extra cov element for background purpose
    return k,intial_weights,means,cov

In [7]:
class data_objects:
    def __init__(self,clip_no,stime,etime,weights_data,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.weights_data = weights_data
        self.label = label

In [8]:
import numpy
from scipy.special import gammaln

class DefaultDict(dict):
    def __init__(self, v):
        self.v = v
        dict.__init__(self)
    def __getitem__(self, k):
        return dict.__getitem__(self, k) if k in self else self.v
    def update(self, d):
        dict.update(self, d)
        return self

class HDPLDA:
    def __init__(self, alpha, beta, gamma, docs, V):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.V = V
        self.M = len(docs)

        # t : table index for document j
        #     t=0 means to draw a new table
        self.using_t = [[0] for j in range(self.M)]

        # k : dish(topic) index
        #     k=0 means to draw a new dish
        self.using_k = [0]

        self.x_ji = docs # vocabulary for each document and term
        self.k_jt = [numpy.zeros(1 ,dtype=int) for j in range(self.M)]   # topics of document and table
        self.n_jt = [numpy.zeros(1 ,dtype=int) for j in range(self.M)]   # number of terms for each table of document
        self.n_jtv = [[None] for j in range(self.M)]

        self.m = 0
        self.m_k = numpy.ones(1 ,dtype=int)  # number of tables for each topic
        self.n_k = numpy.array([self.beta * self.V]) # number of terms for each topic ( + beta * V )
        self.n_kv = [DefaultDict(0)]            # number of terms for each topic and vocabulary ( + beta )

        # table for each document and term (-1 means not-assigned)
        self.t_ji = [numpy.zeros(len(x_i), dtype=int) - 1 for x_i in docs]
                    

    def worddist(self):
        """return topic-word distribution without new topic"""
        return [DefaultDict(self.beta / self.n_k[k]).update(
            (v, n_kv / self.n_k[k]) for v, n_kv in self.n_kv[k].items())
                for k in self.using_k if k != 0]

    def docdist(self):
        """return document-topic distribution with new topic"""

        # am_k = effect from table-dish assignment
        am_k = numpy.array(self.m_k, dtype=float)
        am_k[0] = self.gamma
        am_k *= self.alpha / am_k[self.using_k].sum()

        theta = []
        for j, n_jt in enumerate(self.n_jt):
            p_jk = am_k.copy()
            for t in self.using_t[j]:
                if t == 0: continue
                k = self.k_jt[j][t]
                p_jk[k] += n_jt[t]
            p_jk = p_jk[self.using_k]
            theta.append(p_jk / p_jk.sum())

        return numpy.array(theta)
    
    def dump(self, disp_x=False):
        if disp_x: print("x_ji:", self.x_ji)
        print("using_t:", self.using_t)
        print("t_ji:", self.t_ji)
        print("using_k:", self.using_k)
        print("k_jt:", self.k_jt)
        print("----")
        print("n_jt:", self.n_jt)
        print("n_jtv:", self.n_jtv)
        print("n_k:", self.n_k)
        print("n_kv:", self.n_kv)
        print("m:", self.m)
        print("m_k:", self.m_k)


    def sampling_t(self, j, i):
        """sampling t (table) from posterior"""
        self.leave_from_table(j, i)

        v = self.x_ji[j][i]
        f_k = self.calc_f_k(v)
        assert f_k[0] == 0 # f_k[0] is a dummy and will be erased

        # sampling from posterior p(t_ji=t)
        p_t = self.calc_table_posterior(j, f_k)
        if len(p_t) > 1 and p_t[1] < 0: self.dump()
        t_new = self.using_t[j][numpy.random.multinomial(1, p_t).argmax()]
        if t_new == 0:
            p_k = self.calc_dish_posterior_w(f_k)
            k_new = self.using_k[numpy.random.multinomial(1, p_k).argmax()]
            if k_new == 0:
                k_new = self.add_new_dish()
            t_new = self.add_new_table(j, k_new)

        # increase counters
        self.seat_at_table(j, i, t_new)
        return p_t

    def leave_from_table(self, j, i):
        t = self.t_ji[j][i]
        if t  > 0:
            k = self.k_jt[j][t]
            assert k > 0

            # decrease counters
            v = self.x_ji[j][i]
            self.n_kv[k][v] -= 1
            self.n_k[k] -= 1
            self.n_jt[j][t] -= 1
            self.n_jtv[j][t][v] -= 1

            if self.n_jt[j][t] == 0:
                self.remove_table(j, t)

    def remove_table(self, j, t):
        """remove the table where all guests are gone"""
        k = self.k_jt[j][t]
        self.using_t[j].remove(t)
        self.m_k[k] -= 1
        self.m -= 1
        assert self.m_k[k] >= 0
        if self.m_k[k] == 0:
            # remove topic (dish) where all tables are gone
            self.using_k.remove(k)

    def calc_f_k(self, v):
        return [n_kv[v] for n_kv in self.n_kv] / self.n_k

    def calc_table_posterior(self, j, f_k):
        using_t = self.using_t[j]
        p_t = self.n_jt[j][using_t] * f_k[self.k_jt[j][using_t]]
        p_x_ji = numpy.inner(self.m_k, f_k) + self.gamma / self.V
        p_t[0] = p_x_ji * self.alpha / (self.gamma + self.m)
        #print("un-normalized p_t = ", p_t)
        return p_t / p_t.sum()

    def seat_at_table(self, j, i, t_new):
        assert t_new in self.using_t[j]
        self.t_ji[j][i] = t_new
        self.n_jt[j][t_new] += 1

        k_new = self.k_jt[j][t_new]
        self.n_k[k_new] += 1

        v = self.x_ji[j][i]
        self.n_kv[k_new][v] += 1
        self.n_jtv[j][t_new][v] += 1

    # Assign guest x_ji to a new table and draw topic (dish) of the table
    def add_new_table(self, j, k_new):
        assert k_new in self.using_k
        for t_new, t in enumerate(self.using_t[j]):
            if t_new != t: break
        else:
            t_new = len(self.using_t[j])
            self.n_jt[j].resize(t_new+1)
            self.k_jt[j].resize(t_new+1)
            self.n_jtv[j].append(None)

        self.using_t[j].insert(t_new, t_new)
        self.n_jt[j][t_new] = 0  # to make sure
        self.n_jtv[j][t_new] = DefaultDict(0)

        self.k_jt[j][t_new] = k_new
        self.m_k[k_new] += 1
        self.m += 1

        return t_new

    def calc_dish_posterior_w(self, f_k):
        "calculate dish(topic) posterior when one word is removed"
        p_k = (self.m_k * f_k)[self.using_k]
        p_k[0] = self.gamma / self.V
        return p_k / p_k.sum()



    def sampling_k(self, j, t):
        """sampling k (dish=topic) from posterior"""
        self.leave_from_dish(j, t)

        # sampling of k
        p_k = self.calc_dish_posterior_t(j, t)
        k_new = self.using_k[numpy.random.multinomial(1, p_k).argmax()]
        if k_new == 0:
            k_new = self.add_new_dish()

        self.seat_at_dish(j, t, k_new)

    def leave_from_dish(self, j, t):
        """
        This makes the table leave from its dish and only the table counter decrease.
        The word counters (n_k and n_kv) stay.
        """
        k = self.k_jt[j][t]
        assert k > 0
        assert self.m_k[k] > 0
        self.m_k[k] -= 1
        self.m -= 1
        if self.m_k[k] == 0:
            self.using_k.remove(k)
            self.k_jt[j][t] = 0

    def calc_dish_posterior_t(self, j, t):
        "calculate dish(topic) posterior when one table is removed"
        k_old = self.k_jt[j][t]     # it may be zero (means a removed dish)
        #print("V=", self.V, "beta=", self.beta, "n_k=", self.n_k)
        Vbeta = self.V * self.beta
        n_k = self.n_k.copy()
        n_jt = self.n_jt[j][t]
        n_k[k_old] -= n_jt
        n_k = n_k[self.using_k]
        log_p_k = numpy.log(self.m_k[self.using_k]) + gammaln(n_k) - gammaln(n_k + n_jt)
        log_p_k_new = numpy.log(self.gamma) + gammaln(Vbeta) - gammaln(Vbeta + n_jt)
        #print("log_p_k_new+=gammaln(",Vbeta,") - gammaln(",Vbeta + n_jt,")")

        gammaln_beta = gammaln(self.beta)
        for w, n_jtw in self.n_jtv[j][t].items():
            assert n_jtw >= 0
            if n_jtw == 0: continue
            n_kw = numpy.array([n.get(w, self.beta) for n in self.n_kv])
            n_kw[k_old] -= n_jtw
            n_kw = n_kw[self.using_k]
            n_kw[0] = 1 # dummy for logarithm's warning
            if numpy.any(n_kw <= 0): print(n_kw) # for debug
            log_p_k += gammaln(n_kw + n_jtw) - gammaln(n_kw)
            log_p_k_new += gammaln(self.beta + n_jtw) - gammaln_beta
            #print("log_p_k_new+=gammaln(",self.beta + n_jtw,") - gammaln(",self.beta,"), w=",w)
        log_p_k[0] = log_p_k_new
        #print("un-normalized p_k = ", numpy.exp(log_p_k))
        p_k = numpy.exp(log_p_k - log_p_k.max())
        return p_k / p_k.sum()

    def seat_at_dish(self, j, t, k_new):
        self.m += 1
        self.m_k[k_new] += 1

        k_old = self.k_jt[j][t]     # it may be zero (means a removed dish)
        if k_new != k_old:
            self.k_jt[j][t] = k_new

            n_jt = self.n_jt[j][t]
            if k_old != 0: self.n_k[k_old] -= n_jt
            self.n_k[k_new] += n_jt
            for v, n in self.n_jtv[j][t].items():
                if k_old != 0: self.n_kv[k_old][v] -= n
                self.n_kv[k_new][v] += n


    def add_new_dish(self):
        "This is commonly used by sampling_t and sampling_k."
        for k_new, k in enumerate(self.using_k):
            if k_new != k: break
        else:
            k_new = len(self.using_k)
            if k_new >= len(self.n_kv):
                self.n_k = numpy.resize(self.n_k, k_new + 1)
                self.m_k = numpy.resize(self.m_k, k_new + 1)
                self.n_kv.append(None)
            assert k_new == self.using_k[-1] + 1
            assert k_new < len(self.n_kv)

        self.using_k.insert(k_new, k_new)
        self.n_k[k_new] = self.beta * self.V
        self.m_k[k_new] = 0
        self.n_kv[k_new] = DefaultDict(self.beta)
        return k_new
    
    def inference(self):
        weights = []
        for j, x_i in enumerate(self.x_ji):
            p_t = []
            for i in range(len(x_i)):
                p_t = self.sampling_t(j, i)
            weights.extend(p_t)
        for j in range(self.M):
            for t in self.using_t[j]:
                if t != 0: self.sampling_k(j, t)
        return np.asarray(weights).flatten()
    
    def hdplda_learning(self,iteration):
        for i in range(iteration):
            weights = self.inference()
        return weights       

In [9]:
#Function for distribution formation from the mag array of a clip
def func_distribution_formation(arr):
    print(arr[0].shape) 
    n,height,width=arr[0].shape
    dist_arr_arr = []
    for k in range(n):
        dist_arr =[]
        for idx in range(int(height)):
            for j in range(int(width)):
                dist=[]
                for i in range(len(arr)):
                    dist.append(arr[i][k][idx,j])
                dist=np.asarray(dist)
                dist_arr.append(dist)
        dist_arr_arr.append(dist_arr)
    dist_arr_arr=np.asarray(dist_arr_arr)
    print("shape ",dist_arr_arr.shape)
    return dist_arr_arr

In [10]:
def region_formation(mag_arr):
    regions_arr = []
    shape = mag_arr[0].shape
    x_val = shape[0]//3
    y_val = shape[1]//3
    for i in range(len(mag_arr)):
        regions_arr.append((mag_arr[i].reshape(int(shape[0]/x_val), x_val, -1, y_val).swapaxes(1,2).reshape(-1, x_val, y_val)))
    return regions_arr

In [11]:
#Function to run video input,distribution formation and input to GMM and get updated weights as output
def run(path,ann_file_path):
    mag_arr_all_clips,loa = video_input(path,ann_file_path)
    k,initial_weights,means,cov = intial_parameters(mag_arr_all_clips[0])
    data_object_array=[]
    regions_for_all_clips = []
    dist_arr_all_clips = []
    for index in range(len(mag_arr_all_clips)):
        regions_for_all_clips.append(region_formation(mag_arr_all_clips[index]))
    print(len(regions_for_all_clips))
    print(len(regions_for_all_clips[0]))
    for index in range(len(regions_for_all_clips)):
        dist_arr_all_clips.append(func_distribution_formation(regions_for_all_clips[index]))
    for i in range(len(dist_arr_all_clips)):
        print("clip no ",i)
        clip_weights = []
        for j in range(len(dist_arr_all_clips[i])):
            print("region no ",j)
            hdplda = HDPLDA(1,1,1, dist_arr_all_clips[i][j],len(dist_arr_all_clips[0][0][0]))
            weights = hdplda.hdplda_learning(3)
            clip_weights.extend(weights)
        data_object_array.append(data_objects(loa[i].clip_no,loa[i].stime,loa[i].etime,np.asarray(clip_weights).flatten(),loa[i].label)) 
        print(len(clip_weights))
    return data_object_array

In [12]:
path = "E:\\Study\\Sem Project\\Data\\traffic-junction.avi"
ann_file_path = "E:\\Study\\Sem Project\\Data\\abnormal_times.txt"
total_data_objects = run(path,ann_file_path)

previous shape  (24, 30)
Total no of clips 1327
total frames 66324
1327
49
(9, 8, 10)
shape  (9, 80, 49)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80,

shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9,

shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9,

shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9,

shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9, 80, 50)
(9, 8, 10)
shape  (9,

...

region no  5
region no  6
region no  7
region no  8
3877
clip no  1045
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3751
clip no  1046
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3796
clip no  1047
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3799
clip no  1048
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3825
clip no  1049
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3746
clip no  1050
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3759
clip no  1051
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no

region no  8
3829
clip no  1105
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3798
clip no  1106
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3821
clip no  1107
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3721
clip no  1108
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3695
clip no  1109
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3763
clip no  1110
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3826
clip no  1111
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3785
clip no  1112
region no  0
reg

region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3835
clip no  1166
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3841
clip no  1167
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3750
clip no  1168
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3867
clip no  1169
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3711
clip no  1170
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3829
clip no  1171
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3804
clip no  1172
region no  0
region no  1
region no  2
region no  3
region no

region no  4
region no  5
region no  6
region no  7
region no  8
3752
clip no  1226
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3729
clip no  1227
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3732
clip no  1228
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3715
clip no  1229
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3792
clip no  1230
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3843
clip no  1231
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3828
clip no  1232
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no

region no  7
region no  8
3828
clip no  1286
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3747
clip no  1287
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3798
clip no  1288
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3840
clip no  1289
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3787
clip no  1290
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3813
clip no  1291
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3763
clip no  1292
region no  0
region no  1
region no  2
region no  3
region no  4
region no  5
region no  6
region no  7
region no  8
3782
clip no  1293
reg

In [13]:
print("total no of objects",len(total_data_objects))
for i in range(5):
    print("clip no=",total_data_objects[i].clip_no)
    print("start time=",total_data_objects[i].stime)
    print("end time=",total_data_objects[i].etime)
    print("weights =",total_data_objects[i].weights_data)
    print("label = ",total_data_objects[i].label,"\n")

total no of objects 1327
clip no= 1327
start time= 0:44:12
end time= 0:44:13
weights = [0.01888867 0.43213707 0.24693547 ... 0.43296211 0.06525018 0.08862337]
label =  -1 

clip no= 1327
start time= 0:44:12
end time= 0:44:13
weights = [0.01810735 0.15675295 0.3362437  ... 0.28727857 0.29446053 0.39702544]
label =  -1 

clip no= 1327
start time= 0:44:12
end time= 0:44:13
weights = [0.02052948 0.5122491  0.14924855 ... 0.5443531  0.07302298 0.36355011]
label =  -1 

clip no= 1327
start time= 0:44:12
end time= 0:44:13
weights = [0.01894811 0.49243422 0.21963915 ... 0.04299025 0.27555195 0.10339427]
label =  -1 

clip no= 1327
start time= 0:44:12
end time= 0:44:13
weights = [0.02130355 0.65023463 0.17007732 ... 0.15418828 0.1670373  0.02231296]
label =  -1 



In [14]:
import csv
#writing into the csv file
with open("E:\\Study\\Sem Project\\Data\\hdp_data.csv",'w',newline='') as file:
    writer = csv.writer(file)
    for i in range(len(total_data_objects)):
        llist=[]
        llist.append(total_data_objects[i].clip_no)
        llist.append(total_data_objects[i].stime)
        llist.append(total_data_objects[i].etime)
        llist.append(total_data_objects[i].label)
        for j in range(len(total_data_objects[i].weights_data)):
            llist.append(total_data_objects[i].weights_data[j])
        writer.writerow(llist)